# Exploring duckdb, making plots, and messing around with SQL

In [1]:
# dependecies
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import scipy.stats

# database (make sure to install this. I will make an virtual enviroment for all the dependecies later)
import duckdb

In [3]:
# seaborn plot configurations
sns.set_context('talk')
#sns.set_style('whitegrid')
palette = sns.color_palette('colorblind')

#### Exploring duck

In [4]:
# establishing connection to the database (from local repo)
con = duckdb.connect('/Users/humoodalanzi/learn2therm/data/database')

In [5]:
# what is this connection? (somekind of special object)
con

In [6]:
# creating a table that has my protein pairs as well as my taxa pairs (check schema table shared in Slack)
test = con.execute("""
    SELECT 
        protein_pairs.prot_pair_index,
        proteins_m.protein_seq AS meso_seq,
        proteins_t.protein_seq AS thermo_seq,
        taxa_m.ogt AS meso_ogt,
        taxa_t.ogt AS thermo_ogt,
        protein_pairs.scaled_local_symmetric_percent_id
    FROM protein_pairs
    INNER JOIN taxa AS taxa_m ON (protein_pairs.meso_index=taxa_m.taxa_index)
    INNER JOIN taxa AS taxa_t ON (protein_pairs.thermo_index=taxa_t.taxa_index)
    INNER JOIN proteins AS proteins_m ON (protein_pairs.meso_protein_int_index=proteins_m.protein_int_index)
    INNER JOIN proteins AS proteins_t ON (protein_pairs.thermo_protein_int_index=proteins_t.protein_int_index)
    WHERE
        taxa_m.len_16s>1300
        AND taxa_t.len_16s>1300
        AND protein_pairs.local_E_Value< 1e-46
        AND protein_pairs.query_align_cov > 0.95
        AND protein_pairs.subject_align_cov > 0.95
        AND protein_pairs.scaled_local_symmetric_percent_id > 0.7
""").df()

In [7]:
test #dataframe output

,prot_pair_index,meso_seq,thermo_seq,meso_ogt,thermo_ogt,scaled_local_symmetric_percent_id
0,173543228,MDIDGLRGRLTPNRALADLTWLRVGGPADYLFQPADIEDLSHFMRA...,MTTLPTPRGPLTPNRPLADLTWLRVGGPADWLFQPADEADLAQFLA...,24.0,42.5,0.734959
1,173547174,MTDGMLDMNAKPTESISVRDVFGIDTDMTIKGFAEGSDRVPALDPT...,MLDQVMKPTEEISVRDVFGIDTDMRVKGFADRTDRVPEIDPTYKFD...,24.0,42.5,0.889571
2,15218012,MLTFTIRRLILAVPTLLFISLVIFLLLELAPGDPMAQVPLTVPPEV...,MLNYTIRRLILAVPTLLFIALTIFLLLELAPGDPVAHMPLTIPPEV...,24.0,54.0,0.812772
3,173552217,MTVFAFYLFAISAITGGLFTVISRQPVHSVLWLILAFISSAGLFVL...,MTVFSFAFYMFAVITVTAGLMVTLSRNPVHSVLWLILAFLSTSGLF...,24.0,42.5,0.727723
4,173552221,MLLAVDCGNTNTVFSIWDGENFVGTWRTATDWRRTADQYYVWLNTL...,MLLAIDCGNTNTVFSIWDGARFLATWRIATDHKRTADEYFVWLSSL...,24.0,42.5,0.719536
...,...,...,...,...,...,...
1738489,133632443,MSDAKLEIQMLHDRVMVKLNPEAGERRSSGGILIPATAQVAKRLSW...,MSEPKFEVQMLHDRVMIKVEQESGERRSSGGIVIPATVRQSNRLAW...,28.0,52.5,0.707547
1738490,58837791,MKYSGWPDSGELVVGEVDEITDFGVFVDLEEYEDKRGLCHISEVAN...,MKYEGWPEPGELVVGKVDEIEDFGVFVDLEEYRDKRGLVHISEVAS...,30.0,42.5,0.726592
1738491,43501488,MTHVVTENCIKCRYTDCVDVCPVDCFREGPNFLAIDPDECIDCAVC...,MTHVVTENCINCKYTDCVDVCPVDCFKEGPNFLVIDPDECIDCAVC...,30.0,48.0,0.768519
1738492,143509651,MTHIVTEACIKCKYTDCVDVCPVDCFREGPNFLTIDPDECIDCAVC...,MTHVVTENCINCKYTDCVDVCPVDCFKEGPNFLVIDPDECIDCAVC...,25.0,48.0,0.779817
